In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [55]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)
tX = np.c_[np.ones(X.shape[0]),X]
print(len(y))
print(len(y[y > 0]))

250000
85667


In [56]:
pri0_to_drop = ["DER_deltaeta_jet_jet","DER_mass_jet_jet","DER_prodeta_jet_jet","DER_lep_eta_centrality","PRI_jet_leading_pt","PRI_jet_leading_eta","PRI_jet_leading_phi","PRI_jet_subleading_pt","PRI_jet_subleading_eta","PRI_jet_subleading_phi"]
pri1_to_drop = ["DER_deltaeta_jet_jet","DER_mass_jet_jet","DER_prodeta_jet_jet","DER_lep_eta_centrality","PRI_jet_subleading_pt","PRI_jet_subleading_eta","PRI_jet_subleading_phi"]

In [57]:
import pandas as pd
from IPython.display import display

In [67]:
hb = pd.read_csv(DATA_TRAIN_PATH, sep=',')
pd.options.display.max_columns = None
hb = hb.drop(['Id'], 1)
hb.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,158.432217,1.437609,-0.128305,-708.985189,38.707419,-0.010973,-0.008171,46.660207,-0.019507,0.043543,41.717235,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,115.706115,0.844743,1.193585,453.596721,22.412081,1.214079,1.816763,22.064922,1.264982,1.816611,32.894693,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,46.104000,0.047000,-1.414000,-999.000000,20.000000,-2.499000,-3.142000,26.000000,-2.505000,-3.142000,0.109000,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,77.550000,0.883000,-1.371000,-999.000000,24.591750,-0.925000,-1.575000,32.375000,-1.014000,-1.522000,21.398000,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,120.664500,1.280000,-0.356000,-999.000000,31.804000,-0.023000,-0.033000,40.516000,-0.045000,0.086000,34.802000,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,200.478250,1.777000,1.225000,0.000000,45.017000,0.898000,1.565000,53.390000,0.959000,1.618000,51.895000,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,19.773000,1.414000,1.000000,764.408000,2.497000,3.142000,560.271000,2.503000,3.142000,2842.617000,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [68]:
hb = hb.replace(-999, np.nan)

In [28]:
pri0 = hb[hb.PRI_jet_num==0].copy()
pri0 = pri0.drop(pri0_to_drop,1)
pri0 = pri0.drop(["PRI_jet_num","PRI_jet_all_pt"],1)

pri1 = hb[hb.PRI_jet_num == 1].copy()
pri1 = pri1.drop(pri1_to_drop,1)
pri1 = pri1.drop(["PRI_jet_num"],1)

pri2 = hb[hb.PRI_jet_num == 2].copy()
pri2 = pri2.drop(["PRI_jet_num"],1)

pri3 = hb[hb.PRI_jet_num == 3].copy()
pri3 = pri3.drop(["PRI_jet_num"],1)

pri = [pri0,pri1,pri2,pri3]


In [60]:
def cleanDataSet(dataset):
    pri0 = dataset[dataset.PRI_jet_num==0].copy()
    pri0 = pri0.drop(pri0_to_drop,1)
    pri0 = pri0.drop(["PRI_jet_num","PRI_jet_all_pt"],1)

    pri1 = dataset[dataset.PRI_jet_num == 1].copy()
    pri1 = pri1.drop(pri1_to_drop,1)
    pri1 = pri1.drop(["PRI_jet_num"],1)

    pri2 = dataset[dataset.PRI_jet_num == 2].copy()
    pri2 = pri2.drop(["PRI_jet_num"],1)

    pri3 = dataset[dataset.PRI_jet_num == 3].copy()
    pri3 = pri3.drop(["PRI_jet_num"],1)
    
    return [pri0,pri1,pri2,pri3]

In [61]:
def extractPredictions(dataset):
    return dataset.Prediction.apply(lambda x: -1 if x == 'b' else 1)

In [62]:
def normalizeDataset(dataset):
    dataset = (dataset - dataset.mean()) / dataset.std()
    dataset = dataset.fillna(0)
    dataset = (dataset - dataset.mean()) / dataset.std()
    return dataset

In [63]:
def addX_0_constant(dataset):
    return np.c_[np.ones(dataset.shape[0]), dataset.to_numpy()]

In [69]:
pri = cleanDataSet(hb)
predictions = []

for idx, dataset in enumerate(pri):
    predictions.append(extractPredictions(dataset))
    dataset = dataset.drop(['Prediction'],1)
    pri[idx] = normalizeDataset(dataset)
pri[0].describe()

In [7]:
tX = np.c_[np.ones(X.shape[0]), hb.to_numpy()]

In [71]:
def polynomial_expansion(x, degree = 2):
    """
    Do a polynomial expansion of matrix x up to degree
    The minimum degree is 2.
    
    Parameters
    ----------
    x: ndarray
        feature matrix x
    degree: int
        highest degree of the expansion matrix
    
    Returns
    -------
    ndarray
        x expanded up to degree
    """
    def poly(x, degree):
        """
        polynomial basis functions for input data x, for j=2 up to j=degree.
        """
        acc = x*x
        phi = acc.copy()
        for i in range(2, degree):
            acc *= x
            phi = np.c_[phi, acc]
        return phi

    for i in range(0,x.shape[1]):
        x = np.c_[x, np.apply_along_axis(poly , 0, x[:,i], degree)]
    return x

polynomial_expansion(np.array([[2,3],[2,3]]),4)

array([[ 2,  3,  4,  8, 16,  9, 27, 81],
       [ 2,  3,  4,  8, 16,  9, 27, 81]])

## Do your thing crazy machine learning thing here :) ...

In [8]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MSE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss
'''
weights = np.array([])
for i in range(100):
    initial_w = np.full(tX.shape[1], i/100)
    max_iters = 100
    gamma = 0.3
    w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
    weights = np.append(weights, loss)
idx = np.argmin(weights)
'''
initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.3404106662758695


In [9]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n, e):
        return 1/n * np.sum(np.abs(e))
    
    def compute_gradient(tx, n, e):
        e = y - tx @ w
    
        return -1/n*tx.T @ np.sign(e)
    
    loss = 0
    w = initial_w
    n = y.shape[0]
    n2 = n*2
    
    for n_iter in range(max_iters):
        e = compute_e(y, tx, w)
        gradient = compute_gradient(tx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))

    return w, loss

initial_w = np.zeros(tX.shape[1])
max_iters = 1000
gamma = 0.3
w, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.34672967168835184


In [10]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """
    Linear regression using stochastic gradient descent.
    Uses MAE.
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term
    initial_w: ndarray
        initial weight vector
    max_iters: int
        maximum number of iterations
    gamma: float
        learning rate

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """
    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    def compute_gradient(tx, n, e):
        return - tx.T @ e / n
    
    loss = 0
    w = initial_w[:, np.newaxis]
    n = y.shape[0]
    n2 = n*2
    data_size = len(y)
    shuffled_indices = np.random.permutation(np.arange(data_size))
    shuffled_y = y[shuffled_indices]
    shuffled_tx = tx[shuffled_indices]
    shuffled_y = shuffled_y[:,np.newaxis]
    for n_iter, by, btx in zip(range(max_iters), shuffled_y, shuffled_tx):
        by = by[np.newaxis]
        btx = btx[np.newaxis, :]
        e = compute_e(by, btx, w)
        gradient = compute_gradient(btx, n, e)
        loss = compute_loss(n2, e)
        
        # Update weights
        w -= gamma * gradient
        #print("Gradient Descent({bi}/{ti}): loss={l}, w={w}".format(
        #      bi=n_iter, ti=max_iters - 1, l=loss, w=w[0]))
    return w, compute_loss(n2, compute_e(y, tx, w[:,0]))

initial_w = np.full(tX.shape[1], 0.1)
max_iters = 100000
gamma = 0.7
w, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
print(loss)

0.4915113296368542


In [11]:
def least_squares(y, tx):
    """
    Linear regression using normal equations.
    Use MSE loss function
    
    Parameters
    ----------
    y:  ndarray
        the labels
    tx: ndarray
        vector x tilde, i.e. the parameters with a bias term

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    w = la.solve(tx.T @ tx, tx.T @ y)
    
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))

w, loss = least_squares(y, tX)
print(loss)

0.34040945216155494


In [12]:
def ridge_regression(y, tx, lambda_):
    """
    Ridge regression using normal equations.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """ 
    
            
    def compute_e(y, tx, w):
        return y - tx @ w
    
    def compute_loss(n2, e):
        return (e.T @ e) / n2
    
    
    X = tx.T @ tx
    w = la.solve(X + lambda_ * (2*y.shape[0]) * np.eye(X.shape[0]), tx.T @ y)
    return w, compute_loss(y.shape[0]*2, compute_e(y, tx, w))



In [13]:
def ridge_regression_demo(tx, y):

    
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    
    rmse_tr = []
    for ind, lambda_ in enumerate(lambdas):
        weights, loss = ridge_regression(y, tx, lambda_)
        rmse_tr.append(np.sqrt(loss))

        print("lambda={l:.3f}, Training RMSE={tr:.3f}".format(l=lambda_, tr=rmse_tr[ind]))
        

ridge_regression_demo(tX, y)

w, loss = ridge_regression(y, tX, 0.001)
print(loss)

lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.000, Training RMSE=0.583
lambda=0.001, Training RMSE=0.583
lambda=0.001, Training RMSE=0.583
lambda=0.003, Training RMSE=0.583
lambda=0.007, Training RMSE=0.584
lambda=0.016, Training RMSE=0.584
lambda=0.037, Training RMSE=0.586
lambda=0.085, Training RMSE=0.590
lambda=0.193, Training RMSE=0.598
lambda=0.439, Training RMSE=0.613
lambda=1.000, Training RMSE=0.634
0.3404162668067664


In [ ]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return 1/(1 + np.exp(-t))
    
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    return np.sum(np.log(1 + np.exp(tx @ w)) - y * (tx @ w))


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    return tx.T @ (sigmoid(tx@w) - y)


def logistic_regression_step(y, tx, w):
    """return the loss, gradient"""
    return calculate_loss(y, tx, w), calculate_gradient(y, tx, w)
  

In [ ]:
# TODO
def logistic_regression(y, tx, lambda_):
    """
    Logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """  
    def learning_by_gradient_descent(y, tx, w, gamma):
        """
        Do one step of gradient descen using logistic regression.
        Return the loss and the updated w.
        """
        loss = calculate_loss(y, tx, w)
        gradient = calculate_gradient(y,tx,w)
        w = w - gamma * gradient
        return loss, w
    
    # init parameters
    max_iter = 10000
    threshold = 1e-8
    #gamma = 0.01
    losses = []

    # build tx
    w = np.zeros((tx.shape[1], 1))
    y = y[:,np.newaxis]

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
     
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return w, losses[-1]
    

In [ ]:
lambda_ = 1e-8
w, loss = logistic_regression(y, tX, lambda_)

In [ ]:
# TODO
def reg_logistic_regression(y, tx, lambda_):
    """
    Regularized logistic regression using gradient descent or SGD.
    
    Parameters
    ----------
    y : ndarray
        Description of y
    ...

    Returns
    -------
    (ndarray, float)
        Last weight vector and the corresponding loss value
    """    
    def penalized_logistic_regression(y, tx, w, lambda_):
        """return the loss, gradient"""
        loss, gradient = logistic_regression_step(y, tx, w)
        loss     += 2 * lambda_ * la.norm(w)**2
        gradient += lambda_ * w

        return loss, gradient
    
    def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
        """
        Do one step of gradient descent, using the penalized logistic regression.
        Return the loss and updated w.
        """
        loss, gradient = penalized_logistic_regression(y, tx, w, lambda_) 
        w = w - gamma * gradient 

        return loss, w
    
    # init parameters
    max_iter = 10000
    gamma = 1e-8
    threshold = 1e-8
    losses = []

    w = np.zeros((tx.shape[1], 1))
    y = y[:,np.newaxis]

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    

In [ ]:
lambda_ = 0.000001
w, loss = reg_logistic_regression(y, tX, lambda_)

## Generate predictions and save ouput in csv format for submission:

In [42]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)
hbt = pd.read_csv(DATA_TEST_PATH, sep=',')

#hbt = hbt.drop(['Id', 'Prediction'], 1)

In [43]:
hbt = hbt.drop(['Prediction'], 1)

In [44]:
hbt = hbt.set_index(['Id'])

In [53]:
test_pri = []
for i in range(4):
    test_pri.append(hbt[hbt.PRI_jet_num==i])
# TODO finish me. Clean the data with the same methodes as with hb  

In [31]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)
hbt = pd.read_csv(DATA_TEST_PATH, sep=',')
hbt = hbt.drop(['Id', 'Prediction'], 1)
hbt = hbt.replace(-999, np.nan)
hbt = (hbt - hbt.mean()) / hbt.std()
hbt = hbt.fillna(0)
hbt = (hbt - hbt.mean()) / hbt.std()
tX_test = np.c_[np.ones(X_test.shape[0]), hbt.to_numpy()]

In [ ]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w[1:], tX_test)#[:, [0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]]) # Selected desired columns
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [ ]:
print(len(y_pred))
print(len(y_pred[y_pred > 0]))

# Submissions scores
Best score by technique

<ul>
    <li>MSE, gradient descent : 0.649</li>
    <li>MAE, gradient descent : 0.678 </li>
    <li>ridge regression      : 0.664</li>
</ul>
Best score after not being stupid with bias:

* MSE, GD: 
* MAE, GD: 0.639
* LSQ: 0.706
* R-REG: 0.730

Best score after normalizing test set + putting zero where unknown:

* LSQ: 0.747
* R-REG: 0.745

Feature expansion?